# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#total number of rows 
print(len(full_data_rows_list))
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
    

6821


# Part II. Completeing the Apache Cassandra coding portion of the project. 
The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing Apache Cassandra code:

#### Creating a Cluster

In [5]:
#connection to a Cassandra 

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#Createing a Keyspace
try:
    session.execute ("""
    CREATE KEYSPACE IF NOT EXISTS shorog
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}"""
)
except Exception as e :
    print(e)



#### Set Keyspace

In [7]:
#Setting KEYSPACE to the keyspace specified above
session.set_keyspace('shorog')

### Now we need to create tables to run the following queries.With Apache Cassandra, we model the database tables on the queries that we want to run.

## Our queries are: 

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [50]:
## The query will be : SELECT * FROM songs_in_session WHERE sessionId = 338 AND itemInSession = 4
##According to the above query we will make the sessionID as partition key and itemInSession as 
##Clustring key and the both will uniqly identify each row as the PRIMARY KEY 
query = "CREATE TABLE IF NOT EXISTS songs_in_session "
query = query + "(sessionId text,itemInSession int, song_title text, artist_name text, song_length float, PRIMARY KEY(sessionID,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [51]:
#Now we should read the CSV file in order to load the data into the above table. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_in_session (sessionId , itemInSession, song_title, artist_name,song_length)"
        query = query +" VALUES (%s, %s, %s, %s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ##According to the CSV file the indexes are: 
        ##8  ===> sessionId , 3  ===> itemInSession, 9 ===> song_title, 0 ===> artist_name, 5 ===> song_length
        session.execute(query, (line[8], int(line[3]),line[9],line[0],float(line[5])))

#### Verifying our table and insertion by quering the data with SELECT:

In [52]:
## SELECT * FROM songs_in_session WHERE sessionId = 338 AND itemInSession = 4
query = "SELECT * FROM songs_in_session WHERE sessionId = '338' AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
ans = pd.DataFrame(list(rows))
ans.head()

,sessionid,iteminsession,artist_name,song_length,song_title
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [53]:
## The query will be :
#SELECT artist_name, song_title, user_first_name, user_last_name FROM user_session_info WHERE userid = 10 AND sessionid = 182
##According to the above query we will make the sessionid as partition key and userid and itemInSession as Clustring key.
##To reach the session first then the related userIds and sorted by the itemsession. 
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(sessionId text,userId text,itemInSession int, song_title text, artist_name text, song_length float,user_first_name text,user_last_name text , PRIMARY KEY(sessionID,userId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [54]:
#Now we should read the CSV file in order to load the data into the above table. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (sessionId ,userId , itemInSession, song_title, artist_name,song_length,user_first_name,user_last_name)"
        query = query +" VALUES (%s, %s, %s, %s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ##According to the CSV file the indexes are: 
        ##8  ===> sessionId ,10  ===> userId, 3  ===> itemInSession, 9 ===> song_title,
        ##0 ===> artist_name, 5 ===> song_length, 1 ===> first_name, 4 ===> last_name
        session.execute(query, (line[8],line[10], int(line[3]),line[9],line[0],float(line[5]),line[1],line[4]))

In [55]:
##APPLY QUERY to test our work above 
query = "SELECT artist_name, song_title, user_first_name, user_last_name FROM song_playlist_session WHERE userId = '10' AND sessionId = '182' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
ans = pd.DataFrame(list(rows))
ans.head()

,artist_name,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [56]:
## The query will be : SELECT artist_name, song_title, user_first_name, user_last_name FROM user_session_info WHERE userid = 10 AND sessionid = 182
##According to the above query we will make the song_title as partition key and userId as Clustring key by this we will uniquly identify 
##the users by searching by the song name.  
query = "CREATE TABLE IF NOT EXISTS song_listeners "
query = query + "(song_title text,userId text,user_first_name text,user_last_name text , PRIMARY KEY(song_title,userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

                    

In [57]:
#Now we should read the CSV file in order to load the data into the above table. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listeners( song_title, userId,user_first_name,user_last_name)"
        query = query +" VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ##According to the CSV file the indexes are: 
        ## 10  ===> userId,9 ===> song_title, 1 ===> first_name, 4 ===> last_name
        session.execute(query, (line[9],line[10], line[1],line[4]))

In [58]:
##APPLY QUERY 
query = "SELECT  user_first_name, user_last_name FROM song_listeners WHERE song_title= 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
ans = pd.DataFrame(list(rows))
ans.head()

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [59]:

query = "drop table songs_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [60]:
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [61]:
query = "drop table song_listeners"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [62]:
session.shutdown()
cluster.shutdown()